In [1]:
import sqlite3
import numpy as np
from collections import namedtuple

In [2]:
# Create connection
conn = sqlite3.connect('/tmp/AA_db.sqlite')
cur = conn.cursor()

In [7]:
# Create table
cur.execute('CREATE TABLE tiles (coord_x INTEGER, coord_y INTEGER, slide VARCHAR, center_tumor INTEGER, tile_tumor REAL)')

In [9]:
for _ in range(100):
    # Insert 10,000 entries to the table
    slides = ['slide_{}'.format(i) for i in range(10)]
    for _ in range(10000):
        cur.execute('INSERT INTO tiles (coord_x, coord_y, slide, center_tumor, tile_tumor) VALUES (?, ?, ?, ?, ?)',
                   (np.random.randint(low=0, high=500000), np.random.randint(low=0, high=500000), np.random.choice(slides), np.random.randint(low=0, high=2), np.random.rand()))
conn.commit()

In [ ]:
# Create index on SLIDE
cur.execute('CREATE INDEX slide_index ON tiles(slide)')
conn.commit()

In [ ]:
# Create index on LABEL
cur.execute('CREATE INDEX center_index ON tiles(center_tumor)')
conn.commit()

In [ ]:
# Create slide view
cur.execute('CREATE TEMP TABLE slides AS SELECT DISTINCT slide FROM tiles')
conn.commit()

In [103]:
# Create slide count view
cur.execute('CREATE TEMP TABLE counts AS SELECT slide, center_tumor, COUNT(*) AS tile_count FROM tiles GROUP BY slide, center_tumor')
conn.commit()

In [ ]:
def sample():
    label = np.random.randint(low=0, high=2)
    cur.execute('''
        SELECT DISTINCT slide, tile_count
        FROM counts
        WHERE center_tumor = ?
        ORDER BY random()
        LIMIT 1
    ''', (label,))
    slide, tile_count = cur.fetchone()
    offset = np.random.randint(low=0, high=tile_count+1)
    cur.execute('''
        SELECT coord_x, coord_y, center_tumor
        FROM tiles
        WHERE center_tumor=? AND slide=?
        LIMIT 1
        OFFSET ?
    ''', (label, slide, offset))
    return cur.fetchone()

In [132]:
sample()

(229507, 241954, 0)

In [34]:
def get_random_coord(label, slide):
    cur.execute('''
        SELECT coord_x, coord_y, center_tumor
        FROM tiles
        WHERE center_tumor=? AND slide=?
        ORDER BY random()
        LIMIT 1
    ''', (label, slide))
    return cur.fetchall()
get_random_coord(1, 'slide_5')

[(43007, 134050, 1)]

In [55]:
x = cur.execute('''
        SELECT coord_x, coord_y, center_tumor
        FROM tiles
        WHERE center_tumor=? AND slide=?
    ''', (1, 'slide_5'))

In [74]:
x.__next__()

(68849, 57472, 1)

In [ ]:
Slide = namedtuple('Slide', 'slide_name')

In [5]:
conn.close()